In [ ]:
# Cell 1: import
from tetris_utils import TetrisEnv, preprocess_state
import torch
import torch.nn as nn
import numpy as np

In [ ]:
# Cell 2: 환경 초기화
env = TetrisEnv()
state = preprocess_state(env.reset())
done = False
total_score = 0

In [ ]:
# Cell 3: 모델 정의 및 로딩
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128), nn.ReLU(),
            nn.Linear(128, 128), nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    def forward(self, x):
        return self.net(x)

model = DQN(input_dim=env.state_size, output_dim=env.action_size)
model.load_state_dict(torch.load("model/dqn_tetris_model.pth", map_location='cpu'))
model.eval()

In [ ]:
# Cell 4: DQN으로 플레이
while not done:
    state_tensor = torch.FloatTensor(state).unsqueeze(0)
    with torch.no_grad():
        q_values = model(state_tensor)
    action_index = torch.argmax(q_values).item()
    action = env.get_action_from_index(action_index)
    next_state, reward, done = env.step(action)
    env.render()
    state = preprocess_state(next_state)
    total_score += reward

print(f"Total Score (DQN): {total_score}")